## Description of data.csv

data.csv is exactly identical to the storage_data.csv file from the [locuslab/e2e-model-learning](https://github.com/locuslab/e2e-model-learning/blob/2bf43848ac26e25321fea7bfeddf2a24135aa192/battery_storage/storage_data.csv) GitHub repo, which contains the code and data from the "Task-based End-to-end Model Learning in Stochastic Optimization" (Donti et al., 2017) paper.

data.csv has 4 columns:

`datetime`
- type: datetime
- hourly UTC datetime values from 2011-01-03 00:00:00 (UTC) to 2016-12-31 23:00:00 (UTC)
- no missing or NaN values
- because everything is UTC, we don't have to worry about daylight savings

`da_price`
- type: float
- hourly day-ahead energy prices from PJM ("System Energy Price Day Ahead"), given to 2 decimals
- units are most likely $/MWh
- no missing or NaN values
- see https://dataminer2.pjm.com/feed/rt_da_monthly_lmps (accessed on 2024-05-11)

`load_forecast`
- type: float
- hourly load forecast for the "RTO" Forecast Area, in MW
- forecasts are evaluated at 15:45 UTC time on the previous day. For example, if `datetime` is `2016-05-05 18:00:00`, then `load_forecast` is the forecast generated at `2016-05-04 15:45` (UTC) for `2016-05-05 18:00:00`.
- no missing or NaN values
- see https://dataminer2.pjm.com/feed/load_frcstd_hist (accessed on 2024-05-11)

`temp_dca`
- type: float
- temperature in Farenheit
- missing 40 values
- unclear where/how these values were obtained


We largely follow the data processing pipeline from the locuslab/e2e-model-learning repo, with the following changes:
* We keep all price values, including prices > $500/MWh, which Donti et al. marked as outliers.
* We do not include `past temp^2`, `future temp^2`, or `future temp^3` as features.

This notebook processes data.csv and saves labels (energy prices) and standardized features in data.npz. This .npz file has four keys:
- `'X'`: shape [2189, 101], type float64, standardized features
- `'Y'`: shape [2189, 24], type float64, energy prices
- `'X_mean'`: shape [101], type float64, mean of each feature before standardization
- `'X_std'`: shape [101], type float64, std-dev of each feature before standardization

In [ ]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import pytz

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
tz = pytz.timezone('America/New_York')
df = pd.read_csv('data.csv', parse_dates=[0])

display(df)
print(df.dtypes)

# df has 4 columns:
#
# datetime         datetime64[ns]
# da_price                float64
# load_forecast           float64
# temp_dca                float64

In [ ]:
df['log_da_price'] = np.log(df['da_price'])

## Validate and plot data

In [ ]:
def validate_data(df: pd.DataFrame) -> None:
    # verify no missing datetimes
    expected_dts = pd.date_range(datetime(2011, 1, 3, 0, 0, 0), datetime(2016, 12, 31, 23, 0, 0), freq=timedelta(hours=1))
    actual_dts = set(df['datetime'])
    assert len(expected_dts.difference(actual_dts)) == 0

    # verify no NaNs
    for col in ('datetime', 'da_price', 'load_forecast'):
        assert df[col].isna().sum() == 0

    # print number of NaNs in 'temp_dca'
    print('# of NaNs in `temp_dca`:', df['temp_dca'].isna().sum())


validate_data(df)

In [ ]:
def plot_data(df: pd.DataFrame) -> None:
    cols_and_units = (
        ('da_price', '$/MWh'),
        ('log_da_price', 'log($/MWh)'),
        ('load_forecast', 'MW'),
        ('temp_dca', '°F')
    )

    _, axs = plt.subplots(4, 1, figsize=(12, 8), sharex=True, tight_layout=True)
    for i, ax in enumerate(axs):
        col, units = cols_and_units[i]
        ax.plot(df['datetime'], df[col], lw=0.5, color=plt.cm.tab10.colors[i])
        ax.set(ylabel=units)

    _, axs = plt.subplots(1, 4, figsize=(12, 3), tight_layout=True)
    for i, ax in enumerate(axs):
        col, units = cols_and_units[i]
        ax.hist(df[col], bins=100, color=plt.cm.tab10.colors[i])
        ax.set(xlabel=units, ylabel='count', title=col)


plot_data(df)

## Create features

In [ ]:
df['date'] = df['datetime'].dt.date
df['hour'] = df['datetime'].dt.hour

In [ ]:
df_prices = df.pivot(index='date', columns='hour', values='da_price')
df_logprices = df.pivot(index='date', columns='hour', values='log_da_price')
df_load = df.pivot(index='date', columns='hour', values='load_forecast')

In [ ]:
df_temp = df.pivot(index='date', columns='hour', values='temp_dca')
df_temp = df_temp.transpose().bfill().ffill().transpose()

assert df_logprices.index.equals(df_load.index)
assert df_logprices.index.equals(df_temp.index)

In [ ]:
df_dates = df_logprices.index.to_series()

holidays = USFederalHolidayCalendar().holidays(
    start='2011-01-01', end='2017-01-01').date

df_feat = pd.DataFrame({
    'weekend': df_dates.map(lambda x: x.isoweekday() >= 6),
    'holiday': df_dates.isin(holidays),
    'dst': df_dates.map(
        lambda x: tz.localize(datetime.combine(x, datetime.min.time())).dst().seconds > 0
    ),
    "cos_doy": df_dates.map(lambda x: np.cos(x.timetuple().tm_yday/365*2*np.pi)),
    "sin_doy": df_dates.map(lambda x: np.sin(x.timetuple().tm_yday/365*2*np.pi))
})

In [ ]:
# a single training example (x,y) has
# x = [
#     previous day's 24 hourly log-prices,
#     today's 24 hourly load forecast,
#     previous day's 24 hourly temperature,
#     today's 24 hourly temperature,
#     other features about today (is_weekend, is_holiday, is_dst, cos_doy, sin_doy)
# ]
# len(x) = 4 * 24 + 5 = 101

# y = [today's 24 hourly prices]
# len(y) = 24

X = np.hstack([df_logprices.iloc[:-1].values,  # past log-price
               df_load.iloc[1:].values,        # future load forecast
               df_temp.iloc[:-1].values,       # past temp
               df_temp.iloc[1:].values,        # future temp
               df_feat.iloc[1:].values]).astype(np.float64)
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std

Y = df_prices.iloc[1:].values

dates = np.array(df_prices.iloc[1:].index).astype('datetime64[D]')

np.savez_compressed('data.npz', X=X, Y=Y, X_mean=X_mean, X_std=X_std, dates=dates)